## 多种格式数据加载、处理与存储

实际的场景中，我们会在不同的地方遇到各种不同的数据格式（比如大家熟悉的csv与txt，比如网页HTML格式，比如XML格式），我们来一起看看python如何和这些格式的数据打交道。

In [6]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize = (10,6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision = 4)

# 1.各式各样的文本数据¶

1.1 CSV与TXT读取

In [13]:
pd.read_csv('data1.csv')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [15]:
df = pd.read_csv('data1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [19]:
# sep=','  分隔符为逗号
pd.read_table('data1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [22]:
pd.read_csv('data2.csv', header = None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [23]:
pd.read_csv('data2.csv', names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [24]:
names = ['a','b','c','d','message']
pd.read_csv('data2.csv', names = names, index_col = 'message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [26]:
parsed = pd.read_csv('csv_mindex.csv', index_col=['key1','key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [34]:
list(open('data3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [37]:
# sep = '\s+' 分隔符为空格（正则表达式）
result = pd.read_table('data3.txt', sep = '\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [42]:
pd.read_table('data4.csv', header = None)

,0
0,# hey!
1,"a,b,c,d,message"
2,# just wanted to make things more difficult fo...
3,"# who reads CSV files with computers, anyway?"
4,"1,2,3,4,hello"
5,"5,6,7,8,world"
6,"9,10,11,12,foo"


In [40]:
# skiprows=[0,2,3]  跳过0，2 ，4行
pd.read_csv('data4.csv', skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [45]:
pd.read_table('data5.csv', header = None)

,0
0,"something,a,b,c,d,message"
1,"one,1,2,3,4,NA"
2,"two,5,6,,8,world"
3,"three,9,10,11,12,foo"


In [50]:
result = pd.read_csv('data5.csv')
result
# pd.isnull(result)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [52]:
result = pd.read_csv('data5.csv', na_values = ['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [57]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('data5.csv', na_values = sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


1.2 分片/块读取文本数据¶

In [59]:
result = pd.read_csv('data6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


In [61]:
result.head()

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [63]:
#  nrows = 5  读取前五行
pd.read_csv('data6.csv', nrows = 5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [67]:
# chunksize = 100 按照数据块大小读取
chunker = pd.read_csv('data6.csv', chunksize = 100)
chunker

In [71]:
chunker = pd.read_csv('data6.csv', chunksize = 100)

tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value = 0)
    
tot = tot.sort_values(ascending = False)

In [73]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

1.3 把数据写入文本格式¶

In [75]:
data = pd.read_csv('data5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [78]:
data.to_csv('out.csv')
pd.read_csv('out.csv')

,Unnamed: 0,something,a,b,c,d,message
0,0,one,1,2,3.0,4,NaN
1,1,two,5,6,NaN,8,world
2,2,three,9,10,11.0,12,foo


In [81]:
data.to_csv(sys.stdout,sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [83]:
data.to_csv(sys.stdout,na_rep = 'NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [85]:
data.to_csv(sys.stdout, index = False, header = False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [87]:
data.to_csv(sys.stdout, index = False, columns = ['a','b','c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [103]:
dates = pd.date_range('1/1/2000',periods = 7)
# dates
ts = Series(np.arange(7), index = dates)
ts.to_csv('tseries.csv')
# pd.read_csv('tseries.csv',header = None)
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32

In [104]:
Series.from_csv('tseries.csv', parse_dates = True)

C:\Users\QianGuang Lin\Anaconda3\lib\site-packages\pandas\core\series.py:3727: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

1.4 手动读写数据（按要求）

In [108]:
pd.read_table('data7.csv',header = None)

,0
0,"a,""b"",""c"""
1,"1,""2"",""3"""
2,"1,""2"",""3"",""4"""


In [130]:
import csv
import pandas as pd
f = open('data7.csv')

reader = csv.reader(f)
# list(reader)

In [131]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


In [132]:
lines = list(csv.reader(open('data7.csv')))
header, values = lines[0],lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [133]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [134]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [138]:
pd.read_csv('mydata.csv', header = None)

,0
0,one;two;three
1,1;2;3
2,4;5;6
3,7;8;9


1.5 JSON格式的数据

In [156]:
obj = \
"""
{"姓名": "张三",
 "住处": ["天朝", "挖煤国", "万恶的资本主义日不落帝国"],
 "宠物": null,
 "兄弟": [{"姓名": "李四", "年龄": 25, "宠物": "汪星人"},
              {"姓名": "王五", "年龄": 23, "宠物": "喵星人"}]
}
"""

In [157]:
import json
result = json.loads(obj)
result


{'姓名': '张三',
 '住处': ['天朝', '挖煤国', '万恶的资本主义日不落帝国'],
 '宠物': None,
 '兄弟': [{'姓名': '李四', '年龄': 25, '宠物': '汪星人'},
  {'姓名': '王五', '年龄': 23, '宠物': '喵星人'}]}

In [163]:
print(json.dumps(result, ensure_ascii = False))

{"姓名": "张三", "住处": ["天朝", "挖煤国", "万恶的资本主义日不落帝国"], "宠物": null, "兄弟": [{"姓名": "李四", "年龄": 25, "宠物": "汪星人"}, {"姓名": "王五", "年龄": 23, "宠物": "喵星人"}]}


In [165]:
result['兄弟'][0]

{'姓名': '李四', '年龄': 25, '宠物': '汪星人'}

In [166]:
asjson = json.dumps(result)

In [170]:
brothers = DataFrame(result['兄弟'], columns = ['姓名','年龄'])
brothers

,姓名,年龄
0,李四,25
1,王五,23


1.6 人人都爱爬虫，人人都要解析XML 和 HTML¶

In [179]:
from lxml.html import parse
import urllib

paresd = parse(urllib.request.urlopen('https://ask.julyedu.com/'))

doc = paresd.getroot()

In [181]:
doc

<Element html at 0x18716fc86d8>

In [184]:
links = doc.findall('.//a')
links[15:20]

[<Element a at 0x18716fecb38>,
 <Element a at 0x18716fecb88>,
 <Element a at 0x18716fecbd8>,
 <Element a at 0x18716fecc28>,
 <Element a at 0x18716fecc78>]

In [193]:
lnk = links[14]
lnk
lnk.get('href')
print(lnk.text_content())


                        题库


In [194]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
# urls[:]
urls[-10:]

['http://weibo.com/askjulyedu',
 None,
 'https://www.julyedu.com/help/index/about',
 'https://www.julyedu.com/help/index/contact',
 'https://www.julyedu.com/help/index/join',
 'https://ask.julyedu.com/question/55',
 'http://www.miitbeian.gov.cn/',
 'https://tianchi.aliyun.com',
 'https://cloud.tencent.com/developer/edu',
 'https://www.aidaxue.com/?ch=qyzx']

In [196]:
spans = doc.findall('.//span')
len(spans)

123

In [198]:
def _unpack(spans):
    return [val.text_content() for val in spans]

In [201]:
contents = _unpack(spans)
# contents
for content in contents:
    print(content)

NEW
赚奖学金
购物车

 购物车
 通知设置
发起了问题 • 1 人关注 • 0 个回复 • 27 次浏览 • 22 小时前 
				
 •  来自相关主题
发起了问题 • 1 人关注 • 0 个回复 • 32 次浏览 • 1 天前 
				
 •  来自相关主题
贡献
回复了问题 • 6 人关注 • 3 个回复 • 10610 次浏览 • 2 天前				
 •  来自相关主题
贡献
回复了问题 • 2 人关注 • 1 个回复 • 98 次浏览 • 2 天前				
 •  来自相关主题
贡献
回复了问题 • 15 人关注 • 4 个回复 • 3924 次浏览 • 2 天前				
 •  来自相关主题
回复了问题 • 3 人关注 • 1 个回复 • 57 次浏览 • 4 天前				
 •  来自相关主题
贡献
回复了问题 • 3 人关注 • 3 个回复 • 1060 次浏览 • 4 天前				
 •  来自相关主题
贡献
回复了问题 • 13 人关注 • 11 个回复 • 860 次浏览 • 5 天前				
 •  来自相关主题
发起了问题 • 1 人关注 • 0 个回复 • 47 次浏览 • 5 天前 
				
 •  来自相关主题
发起了问题 • 1 人关注 • 0 个回复 • 47 次浏览 • 6 天前 
				
 •  来自相关主题
贡献
回复了问题 • 19 人关注 • 16 个回复 • 1923 次浏览 • 2019-02-20 01:11				
 •  来自相关主题
发起了问题 • 1 人关注 • 0 个回复 • 60 次浏览 • 2019-02-19 15:28 
				
 •  来自相关主题
贡献
回复了问题 • 2 人关注 • 1 个回复 • 64 次浏览 • 2019-02-19 11:33				
 •  来自相关主题
发起了问题 • 2 人关注 • 0 个回复 • 87 次浏览 • 2019-02-18 11:47 
				
 •  来自相关主题
贡献
回复了问题 • 2 人关注 • 1 个回复 • 217 次浏览 • 2019-02-15 15:05				
 •  来自相关主题
贡献
回复了问题 • 2 人关注 • 1 个回复 • 58 次浏览 • 2019-02-15 14:58				
 •  来自相关

In [204]:
questions = doc.findall('.//h4')
len(questions)

50

In [206]:
contents = _unpack(questions)
for content in contents:
    print(content)


						必备收藏！8500+公开代码论文，950多项机器学习任务最优结果汇总
					

						不看后悔！2019年人工智能行业25大趋势
					

						浅谈决策树、GBDT、LightGBM
					

						红包问答：随机森林如何处理缺失值？
					

						面试高频题：给定N个数的间距最大值
					

						自动驾驶课程学员资料共享
					

						【面经】一个机械转行算法的菜逼应届生如何进华为
					

						红包问答（已开奖）：哪些机器学习算法不需要做归一化处理？
					

						再见了，快递员！北京打响第一枪！
					

						必读！2018最具突破性计算机视觉论文Top 10
					

						我的算法岗校招面经：微软、谷歌、阿里、头条、地平线、网易游戏等
					

						特朗普终于顾不得美国人就业，准备举国搞人工智能了
					

						我有门课选错，可以退款吗
					

						程序员面试谈薪指南
					

						悬赏话题：35 岁是不是程序员的一个坎？
					

						春节红包活动
					

						11万份测试告诉你，今年学什么编程语言才能找到好工作
					

						【收藏】机器学习开源框架大汇总，总有一款适合你
					

						如何通俗理解LSTM网络(台大版)
					

						求解答：xgboost 命令行训练和python接口训练得到的模型不一样
					

						nlp笔试题
					

						指南：为什么学ai、ai薪资水平和2019年ai就业前景分析
					

						推荐系统综合项目解析与特征处理
					

						在线推荐系统课期末项目解法和分析
					

						裤子换裙子，就问你GAN的这波操作秀不秀
					

						9元尊享【七月在线VIP年会员】
					

						还在为数据清洗抓狂？这里有一个简单实用的清洗代码集
					

						程序员锁死服务器搅黄游戏，600万打水漂，创始人负债数百万！
					

						全球最厉害的14位程序员！你都认识吗？
					



1.7 解析XML

In [209]:
from lxml import objectify

path = 'Performance_MNR.xml'
paresd = objectify.parse(open(path))
# getroot()  拿到根节点
root = paresd.getroot()
# root[-10:]

[<Element PERFORMANCE at 0x18716ffcdc8>]

In [220]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

#     从root根节点往下找child子节点
#     child.tag孩子节点的标签
for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [221]:
perf = pd.DataFrame(data)
perf

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95
5,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,94.4,95,6,2008,96.2,95
6,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96,95,7,2008,96.2,95
7,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,8,2008,96.2,95
8,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,93.7,95,9,2008,95.9,95
9,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,10,2008,96,95


In [214]:
root

<Element PERFORMANCE at 0x18716ffcdc8>

In [216]:
root.get('href')

In [218]:
root.text

二进制格式的数据¶

In [223]:
frame = pd.read_csv('data1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [225]:
frame.to_pickle('frame_pickle')

In [227]:
pd.read_pickle('frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


使用HDF5格式

In [232]:
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [233]:
store['obj1']

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [234]:
store.close()
os.remove('mydata.h5')

HTML与API交互¶

In [242]:
import requests
url = 'https://api.github.com/repos/pydata/pandas/milestones/28/labels'
response  = requests.get(url)
response 

<Response [200]>

In [244]:
print(response.status_code)  # 打印状态码
print('')
print(response.url)          # 打印请求url
print('')
print(response.headers)      # 打印头信息
print('')
print(response.cookies)      # 打印cookie信息
print('')
print(response.text)  #以文本形式打印网页源码
print('')
print(response.content) #以字节流形式打印
print('')

200

https://api.github.com/repositories/858127/milestones/28/labels

{'Date': 'Thu, 28 Feb 2019 03:23:46 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Server': 'GitHub.com', 'Status': '200 OK', 'X-RateLimit-Limit': '60', 'X-RateLimit-Remaining': '52', 'X-RateLimit-Reset': '1551327569', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding', 'ETag': 'W/"11cba19974e22c4f5be052d38ee22617"', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Link': '<https://api.github.com/repositories/858127/milestones/28/labels?page=2>; rel="next", <https://api.github.com/repositories/858127/milestones/28/labels?page=2>; rel="last"', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=3153

In [245]:
data[:5]

[{'AGENCY_NAME': 'Metro-North Railroad',
  'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.\n',
  'PERIOD_YEAR': 2008,
  'PERIOD_MONTH': 1,
  'CATEGORY': 'Service Indicators',
  'FREQUENCY': 'M',
  'INDICATOR_UNIT': '%',
  'YTD_TARGET': 95.0,
  'YTD_ACTUAL': 96.9,
  'MONTHLY_TARGET': 95.0,
  'MONTHLY_ACTUAL': 96.9},
 {'AGENCY_NAME': 'Metro-North Railroad',
  'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate 

In [247]:
issue_labels = DataFrame(data)
issue_labels

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95
5,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,94.4,95,6,2008,96.2,95
6,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96,95,7,2008,96.2,95
7,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,8,2008,96.2,95
8,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,93.7,95,9,2008,95.9,95
9,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,10,2008,96,95


# 2.数据库相关操作

2.1 sqlite数据库¶

In [248]:
import sqlite3

query = '''
create table test(
    a varchar(20), b varchar(20),
    c real , d integer
);
'''

con = sqlite3.connect(':memory:')  # 创建连接
con.execute(query)  # 执行sql语句
con.commit()  # 提交

In [249]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)  # 执行多行语句
con.commit()

In [252]:
cursor = con.execute('select * from test')
# cursor为游标（类似C的指针）,通过游标cursor选取所有信息cursor.fetchall()
rows = cursor.fetchall()  
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [254]:
cursor.description  # 游标cursor的描述

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [258]:
DataFrame(rows, columns = list(zip(*cursor.description))[0])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [260]:
import pandas.io.sql as sql
sql.read_sql('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


3.2 MySQL数据库¶

In [1]:
# coding = utf-8
import pymysql

# 打开数据库连接
# db = pymysql.connect("localhost","root","root","test",3306)
# db = pymysql.connect("127.0.0.1","root","root","test",3306)
db = pymysql.connect(
    host = 'localhost',
    port = 3306,
    user = 'root',
    passwd = 'root',
    db = 'test01'
)



In [3]:
# 使用预处理语句创建表
sql = """CREATE TABLE EMPLOYEE (
         FIRST_NAME  CHAR(20) NOT NULL,
         LAST_NAME  CHAR(20),
         AGE INT,  
         SEX CHAR(1),
         INCOME FLOAT )"""
 
cursor.execute(sql)
 


0

In [4]:
# 使用 cursor() 方法创建一个游标对象 cursor
cursor  = db.cursor()

# 使用 execute() 方法执行 SQL，如果表存在则删除
cursor.execute("DROP TABLE IF EXISTS EMPLOYEE")
 


0

In [5]:
#创建数据表
cursor.execute("create table student(id int ,name varchar(20),class varchar(30),age varchar(10))")

0

In [6]:
#插入一条数据
cursor.execute("insert into student values('2','Tom','3 year 2 class','9')")

1

In [ ]:
#修改查询条件的数据
cursor.execute("update student set class='3 year 1 class' where name = 'Tom'")

In [ ]:
#删除查询条件的数据
cursor.execute("delete from student where age='9'")

In [9]:
# 关闭数据库连接
# cursor.close()
# db.commit()
# db.close()